# NJ State - Your Legislators

In [1]:
#Import dependencies
import pandas as pd
import os
from sqlalchemy import create_engine
#import psycopg2
from config import db_user,db_password,mayor_query,senator_query,assembly_query

In [2]:
#Connect to DB - User ID and password are retrived from environment variables
url = 'postgresql://{user}:{passwd}@{host}:{port}/{db}'.format(
        user=db_user, passwd=db_password, host="localhost", port=5432, db="NJLegislators_db")
engine = create_engine(url, pool_size = 50)

# Data Extract and Transformation Steps

In [2]:
#Read Dist-Town file
town_district_file= os.path.join("","data","DistrictTowns.csv")
dist_municipality_dfs = pd.read_csv(town_district_file,encoding= 'unicode_escape')

In [3]:
#Display results
dist_municipality_dfs.head(2)

,District Name,Town,County
0,District 13,Aberdeen,Monmouth
1,District 2,Absecon,Atlantic


In [4]:
#Split data 
dist_municipality_dfs["Dist_ID"] = dist_municipality_dfs["District Name"].apply(lambda x: x.split("District ")[1])

In [5]:
#Display results
dist_municipality_dfs.head(5)

,District Name,Town,County,Dist_ID
0,District 13,Aberdeen,Monmouth,13
1,District 2,Absecon,Atlantic,2
2,District 23,Alexandria,Hunterdon,23
3,District 24,Allamuchy,Warren,24
4,District 40,Allendale,Bergen,40


In [6]:
#Select required columns for df
dist_municipality_dfs=dist_municipality_dfs[["Dist_ID","District Name","Town","County"]]

In [7]:
#Rename column
dist_municipality_dfs=dist_municipality_dfs.rename(columns={"District Name":"District"})

In [8]:
#Strip blank spaces
dist_municipality_dfs["Town"]=dist_municipality_dfs["Town"].apply(lambda x: x.strip())
dist_municipality_dfs["County"]=dist_municipality_dfs["County"].apply(lambda x: x.strip())

In [9]:
#Display results-final df
dist_municipality_dfs.head(2)

,Dist_ID,District,Town,County
0,13,District 13,Aberdeen,Monmouth
1,2,District 2,Absecon,Atlantic


In [10]:
#List of columns to create db
dist_municipality_dfs.columns

Index(['Dist_ID', 'District', 'Town', 'County'], dtype='object')

## State Legislators

In [11]:
#Use Pandas to get html table 
legislative_roster_dfs = pd.read_html("https://www.njleg.state.nj.us/members/roster.asp")

In [12]:
#Display results
legislative_roster_dfs[4].head(5)

,0,1
0,District 1,NaN
1,NaN,"Senator MICHAEL L. TESTA, JR. - Republican V..."
2,NaN,"District Office: School House Office Park, 211..."
3,NaN,Assemblyman ANTWAN L. MCCLELLAN - Republican...
4,NaN,"District Office: School House Office Park, 211..."


In [13]:
#Loop through rows and parse info - District, senetor and assembly person

#Lists
district_id=[]
district_name=[]
legislators_title=[]
legislators_name=[]
legislator_offices=[]
party=[]

#Loop
for index, row  in legislative_roster_dfs[4].iterrows():

    try:
        district= str(row[0])
            
        #Check if its district row
        if district.startswith("District"):
            
            districtname=district
            #print("District: ",districtname)
    except:
        pass

    try:
        #Check if its senetor row
        if row[1].startswith("Senator"):
            #Get lesgislator name and party
            legislator=row[1].split("-")[0]
            partys = str(row[1].split("-")[1].strip())
            
            #Check party affiliation
            if partys.startswith("Democrat"):
                party_affiliation="Democrat"
            elif partys.startswith("Republican"):
                party_affiliation="Republican"

            #Get Senetor info
            legislator_position="Senator"
            legislator_name=str(legislator.split("Senator")[1].replace("\xa0"," "))
            
            #Append list
            district_id.append(districtname.replace("District","").strip())
            district_name.append(districtname)
            legislators_title.append(legislator_position)
            legislators_name.append(legislator_name)
            party.append(party_affiliation)
            
        #Check if its assmeblyman row
        if row[1].startswith("Assemblyman"):
            #print("Assembly :",row[1].split("-")[0])
            legislator=row[1].split("-")[0]
            partys = str(row[1].split("-")[1].strip())
            
            #Check party affiliation
            if partys.startswith("Democrat"):
                party_affiliation="Democrat"
            elif partys.startswith("Republican"):
                party_affiliation="Republican"

            legislator_position="Assemblyman"
            legislator_name=str(legislator.split("Assemblyman")[1].replace("\xa0"," "))
            #Append list
            district_id.append(districtname.replace("District","").strip())
            district_name.append(districtname)
            legislators_title.append(legislator_position)
            legislators_name.append(legislator_name)
            party.append(party_affiliation)
            
        #Check if its assmeblywoman row
        if row[1].startswith("Assemblywoman"):
            #print("Assembly :",row[1].split("-")[0])
            legislator=row[1].split("-")[0]
            partys = str(row[1].split("-")[1].strip())
            
            #Check party affiliation
            if partys.startswith("Democrat"):
                party_affiliation="Democrat"
            elif partys.startswith("Republican"):
                party_affiliation="Republican"

            legislator_position="Assemblywoman"
            legislator_name=str(legislator.split("Assemblywoman")[1].replace("\xa0"," ") )  
            #Append list
            district_id.append(districtname.replace("District","").strip())
            district_name.append(districtname)
            legislators_title.append(legislator_position)
            legislators_name.append(legislator_name)
            party.append(party_affiliation)
            
        #Check if its Office address row    
        if row[1].startswith("District Office"):
            #print("District Office :",row[1])
            legislator_office=str(row[1].replace("\xa0"," "))
            #Append list
            legislator_offices.append(legislator_office)
        
        
    except:
        continue

#Build Dataframe        
district_legislator_offices_df = pd.DataFrame({"Dist_ID" :district_id, "District":district_name,"Affiliation":party,
                                                "Position":legislators_title,"Name":legislators_name,
                                                "LegislatorOffice":legislator_offices})
#Display results
district_legislator_offices_df.head(5)

,Dist_ID,District,Affiliation,Position,Name,LegislatorOffice
0,1,District 1,Republican,Senator,"MICHAEL L. TESTA, JR.","District Office: School House Office Park, 211..."
1,1,District 1,Republican,Assemblyman,ANTWAN L. MCCLELLAN,"District Office: School House Office Park, 211..."
2,1,District 1,Republican,Assemblyman,ERIK K. SIMONSEN,"District Office: School House Office Park, 211..."
3,2,District 2,Republican,Senator,CHRIS A. BROWN,"District Office: 2021 New Rd., Suite 24, Linwo..."
4,2,District 2,Democrat,Assemblyman,JOHN ARMATO,"District Office: 2312 New Road, Suite 101, Nor..."


### NJ State Senetor - Legislators

In [14]:
#Let's Seperate Senetors 
senator_df = district_legislator_offices_df[district_legislator_offices_df["Position"]=="Senator"]

In [15]:
#Display results
senator_df.head(2)

,Dist_ID,District,Affiliation,Position,Name,LegislatorOffice
0,1,District 1,Republican,Senator,"MICHAEL L. TESTA, JR.","District Office: School House Office Park, 211..."
3,2,District 2,Republican,Senator,CHRIS A. BROWN,"District Office: 2021 New Rd., Suite 24, Linwo..."


In [16]:
#List of senetors
senator_legislators_df=senator_df[["Dist_ID","Name","Affiliation"]]

In [17]:
#Display results- final df
senator_legislators_df.head(5)

,Dist_ID,Name,Affiliation
0,1,"MICHAEL L. TESTA, JR.",Republican
3,2,CHRIS A. BROWN,Republican
6,3,STEPHEN M. SWEENEY,Democrat
9,4,"FRED H. MADDEN, JR.",Democrat
12,5,NILSA I. CRUZ,Democrat


In [18]:
#List columns to create db table
senator_legislators_df.columns

Index(['Dist_ID', 'Name', 'Affiliation'], dtype='object')

### Senetor office address - by district

In [19]:
#Let's seperate addresses as there are more than one address for a legislator
seneate_office_df =senator_df[["Dist_ID","LegislatorOffice"]]

In [20]:
#Display results
seneate_office_df.head(5)

,Dist_ID,LegislatorOffice
0,1,"District Office: School House Office Park, 211..."
3,2,"District Office: 2021 New Rd., Suite 24, Linwo..."
6,3,"District Office: 935 Kings Highway, Suite 400,..."
9,4,"District Office: 129 Johnson Road, Suite 1, Tu..."
12,5,"District Office: 231-L Market Street, Camden, ..."


In [21]:
#Variable Declaration
Address_Id=[]
Dist_Id=[]
seneate_address=[]
Address_No=[]
iAddress=1
iDist=0
iCounter=0

#Get senetor address info
senetor_address_info=senator_df["LegislatorOffice"].str.split("District Office:")

for item in senetor_address_info:
    #Get temp address
    tmp_address = str(item).split("', '")
    
    #Increment district ID and Counter
    iDist=iDist+1    

    
    if len(tmp_address) == 2:
        #Get address
        seneate_address.append(tmp_address[1].replace("']",""))

        #Increment record count
        iCounter=iCounter+1

        #Add info to list
        Dist_Id.append(iDist)
        Address_No.append(iAddress)
        Address_Id.append(iCounter)

    elif len(tmp_address) == 3:
        #Get first address
        seneate_address.append(tmp_address[1].replace("']",""))

        #Increment record count
        iCounter=iCounter+1

        #Add info to list
        Dist_Id.append(iDist)
        Address_No.append(iAddress)
        Address_Id.append(iCounter)

        #Get 2nd address
        seneate_address.append(tmp_address[2].replace("']",""))

        #Increment record count
        iCounter=iCounter+1
        
        #Add info to list
        Dist_Id.append(iDist)
        Address_No.append(iAddress+1)
        Address_Id.append(iCounter)
       
 
    elif len(tmp_address) == 4:
        
        #Get first address
        seneate_address.append(tmp_address[1].replace("']",""))

        #Increment record count
        iCounter=iCounter+1

        #Add info to list
        Dist_Id.append(iDist)
        Address_No.append(iAddress)
        Address_Id.append(iCounter)

        #Get 2nd address
        seneate_address.append(tmp_address[2].replace("']",""))

        #Increment record count
        iCounter=iCounter+1
        
        #Add info to list
        Dist_Id.append(iDist)
        Address_No.append(iAddress+1)
        Address_Id.append(iCounter)       

        #Get 3rdnd address
        seneate_address.append(tmp_address[3].replace("']",""))

        #Increment record count
        iCounter=iCounter+1
        
        #Add info to list
        Dist_Id.append(iDist)
        Address_No.append(iAddress+2)
        Address_Id.append(iCounter)            

#Add data to DataFrame
seneate_address_df=pd.DataFrame({"Address_ID":Address_Id,"Dist_ID":Dist_Id ,"Add_No":Address_No,"Address":seneate_address})

In [22]:
#Display results-final df
seneate_address_df.head(5)

,Address_ID,Dist_ID,Add_No,Address
0,1,1,1,"School House Office Park, 211 S. Main Street,..."
1,2,2,1,"2021 New Rd., Suite 24, Linwood, NJ 08221 (60..."
2,3,3,1,"935 Kings Highway, Suite 400, West Deptford, ..."
3,4,3,2,"199 East Broadway, 1st Floor, Suite G, Salem..."
4,5,4,1,"129 Johnson Road, Suite 1, Turnersville, NJ 0..."


In [23]:
#List columns to create db table
seneate_address_df.columns

Index(['Address_ID', 'Dist_ID', 'Add_No', 'Address'], dtype='object')

### State Assembly - Legislators

In [24]:
#Let's Seperate Assembly and Office Address
assembly_df = district_legislator_offices_df[district_legislator_offices_df["Position"].str.startswith("Assembly")]
assembly_df.head(5)

,Dist_ID,District,Affiliation,Position,Name,LegislatorOffice
1,1,District 1,Republican,Assemblyman,ANTWAN L. MCCLELLAN,"District Office: School House Office Park, 211..."
2,1,District 1,Republican,Assemblyman,ERIK K. SIMONSEN,"District Office: School House Office Park, 211..."
4,2,District 2,Democrat,Assemblyman,JOHN ARMATO,"District Office: 2312 New Road, Suite 101, Nor..."
5,2,District 2,Democrat,Assemblyman,VINCENT MAZZEO,"District Office: 2312 New Road, Suite 102, Nor..."
7,3,District 3,Democrat,Assemblyman,JOHN J. BURZICHELLI,"District Office: 935 Kings Highway, Suite 400,..."


In [25]:
#Assembly-legislators table
assembly_lesgislators_df=assembly_df[["Dist_ID","Name","Affiliation"]]

In [26]:
#Display results-Final df
assembly_lesgislators_df.tail(5)

,Dist_ID,Name,Affiliation
113,38,P. CHRISTOPHER TULLY,Democrat
115,39,ROBERT AUTH,Republican
116,39,HOLLY T. SCHEPISI,Republican
118,40,CHRISTOPHER P. DEPHILLIPS,Republican
119,40,KEVIN J. ROONEY,Republican


In [27]:
#List columns to create db table
assembly_lesgislators_df.columns

Index(['Dist_ID', 'Name', 'Affiliation'], dtype='object')

### Assembly office address - by district

In [30]:
#Variable Declaration
Address_Id=[]
Dist_Id=[]
assembly_address=[]
Address_No=[]
iAddress=1
iDist=0
iCounter=0
j=2

#Get assembly address info
assembly_info=assembly_df["LegislatorOffice"].str.split("District Office:")

for item in assembly_info:
    #Get temp address
    tmp_address = str(item).split("', '")
    
    #Increment district ID and Counter - there are 40 districts. Added logic to avoid district increment for each row
    j=j+1
    iDist=iDist+ j%2    

    
    if len(tmp_address) == 2:
        #Get address
        assembly_address.append(tmp_address[1].replace("']",""))

        #Increment record count
        iCounter=iCounter+1

        #Add info to list
        Dist_Id.append(iDist)
        Address_No.append(iAddress)
        Address_Id.append(iCounter)

    elif len(tmp_address) == 3:
        #Get first address
        assembly_address.append(tmp_address[1].replace("']",""))

        #Increment record count
        iCounter=iCounter+1

        #Add info to list
        Dist_Id.append(iDist)
        Address_No.append(iAddress)
        Address_Id.append(iCounter)

        #Get 2nd address
        assembly_address.append(tmp_address[2].replace("']",""))

        #Increment record count
        iCounter=iCounter+1
        
        #Add info to list
        Dist_Id.append(iDist)
        Address_No.append(iAddress+1)
        Address_Id.append(iCounter)
       
 
    elif len(tmp_address) == 4:
        
        #Get first address
        assembly_address.append(tmp_address[1].replace("']",""))

        #Increment record count
        iCounter=iCounter+1

        #Add info to list
        Dist_Id.append(iDist)
        Address_No.append(iAddress)
        Address_Id.append(iCounter)

        #Get 2nd address
        assembly_address.append(tmp_address[2].replace("']",""))

        #Increment record count
        iCounter=iCounter+1
        
        #Add info to list
        Dist_Id.append(iDist)
        Address_No.append(iAddress+1)
        Address_Id.append(iCounter)       

        #Get 3rdnd address
        assembly_address.append(tmp_address[3].replace("']",""))

        #Increment record count
        iCounter=iCounter+1
        
        #Add info to list
        Dist_Id.append(iDist)
        Address_No.append(iAddress+2)
        Address_Id.append(iCounter)            

#Add data to DataFrame
assembly_address_df=pd.DataFrame({"Address_ID":Address_Id,"Dist_ID":Dist_Id ,"Add_No":Address_No,"Address":assembly_address})

In [31]:
#Display results - Final df
assembly_address_df.tail(5)

,Address_ID,Dist_ID,Add_No,Address
93,94,39,1,"1069 Ringwood Ave., Suite 312, Haskell, NJ 07..."
94,95,39,2,"350 Madison Ave., Cresskill, NJ 07626 (201) 5..."
95,96,39,1,"287 Kinderkamack Rd., Westwood, NJ 07675 (201..."
96,97,40,1,"201C Franklin Ave., Midland Park, NJ 07432 (2..."
97,98,40,1,"1211 Hamburg Turnpike, Suite 301, Wayne, NJ 0..."


In [32]:
#List columns to create db table
assembly_address_df.columns

Index(['Address_ID', 'Dist_ID', 'Add_No', 'Address'], dtype='object')

## Towns and Mayors

In [33]:
#Read CSV file
filepath= os.path.join("","data","2020mayors.xls")
municipality_mayor_dfs = pd.read_excel(filepath,dtype={'Muni Name':'str',"Mayor Name":"str",'Town':'str','Zip':'str'})

In [34]:
#Display results
municipality_mayor_dfs.head(5)

,Muni Code,Muni Name,Town,County,Address,City,State,Zip,Phone,Fax,Mayor Name,Term Start,Term End,Form,Term Length,eMail,SOCIAL MEDIA HANDLE,Municipal Contact List
0,101,Absecon City,Absecon,Atlantic,Absecon Municipal Complex,Absecon,NJ,8201,(609) 641-0663,(609) 645-5098,KIMBERLY HORTON,NaN,2020-12-31,MAYOR-COUNCIL,3,khorton@abseconnj.org,NaN,NaN
1,102,Atlantic City,Atlantic City,Atlantic,City Hall,Atlantic City,NJ,8401,(609) 347-5300,(609) 437-5638,"FRANK M. GILLIAM, JR.",NaN,2021-12-31,NaN,NaN,pgeletei@CityofatlanticCity.org,NaN,NaN
2,103,Brigantine City,Brigantine,Atlantic,1417 West Brigantine Avenue,Brigantine,NJ,08203-2147,(609) 266-7600,NaN,ANDREW SIMPSON,NaN,2022-12-31,NaN,NaN,asimpson@brigantinebeachnj.com,NaN,NaN
3,105,Buena Vista Township,Buena Vista,Atlantic,890 Harding Highway,Buena,NJ,8310,(856) 697-2100,(856) 697-8353,Teresa Kelly,NaN,2021-12-31,NaN,NaN,tkelly@buenavistanj.com,NaN,NaN
4,106,Corbin City,Corbin City,Atlantic,316 Route 50,Corbin City,NJ,8270,(609) 628-2673,(609) 628-3017,ROBERT SCHULTE,NaN,2020-12-31,NaN,NaN,mayor@corbinCitynj.com,NaN,NaN


In [35]:
#Select required field in df
municipality_mayor_dfs=municipality_mayor_dfs[["Muni Code","Muni Name","Mayor Name","eMail","Address","City","State","Zip","Phone","Fax","Town","County","Term End",]]

In [36]:
#Display results
municipality_mayor_dfs.head(2)

,Muni Code,Muni Name,Mayor Name,eMail,Address,City,State,Zip,Phone,Fax,Town,County,Term End
0,101,Absecon City,KIMBERLY HORTON,khorton@abseconnj.org,Absecon Municipal Complex,Absecon,NJ,8201,(609) 641-0663,(609) 645-5098,Absecon,Atlantic,2020-12-31
1,102,Atlantic City,"FRANK M. GILLIAM, JR.",pgeletei@CityofatlanticCity.org,City Hall,Atlantic City,NJ,8401,(609) 347-5300,(609) 437-5638,Atlantic City,Atlantic,2021-12-31


In [37]:
#Preapre address column
addressln=municipality_mayor_dfs["Address"].apply(lambda x: x.strip())+", " +municipality_mayor_dfs["City"].apply(lambda x: x.strip())+" NJ -" + municipality_mayor_dfs["Zip"].apply(lambda x: x if x.startswith("0") else "0"+x)

In [38]:
#Make address as one field
municipality_mayor_dfs["Address"]=addressln

In [39]:
#Strip blank spaces from columns
municipality_mayor_dfs["eMail"]=municipality_mayor_dfs["eMail"].apply(lambda x: x.strip())
municipality_mayor_dfs["Muni Name"]=municipality_mayor_dfs["Muni Name"].apply(lambda x: x.strip())
municipality_mayor_dfs["Mayor Name"]=municipality_mayor_dfs["Mayor Name"].apply(lambda x: x.strip())
municipality_mayor_dfs["Town"]=municipality_mayor_dfs["Town"].apply(lambda x: x.strip())

In [40]:
#Display result
municipality_mayor_dfs.head(2)

,Muni Code,Muni Name,Mayor Name,eMail,Address,City,State,Zip,Phone,Fax,Town,County,Term End
0,101,Absecon City,KIMBERLY HORTON,khorton@abseconnj.org,"Absecon Municipal Complex, Absecon NJ -08201",Absecon,NJ,8201,(609) 641-0663,(609) 645-5098,Absecon,Atlantic,2020-12-31
1,102,Atlantic City,"FRANK M. GILLIAM, JR.",pgeletei@CityofatlanticCity.org,"City Hall, Atlantic City NJ -08401",Atlantic City,NJ,8401,(609) 347-5300,(609) 437-5638,Atlantic City,Atlantic,2021-12-31


In [41]:
#Select required columns in df 
municipality_mayor_dfs=municipality_mayor_dfs[["Muni Code","Muni Name","Mayor Name","eMail","Address","Phone","Town","County","Term End",]]

In [42]:
#Rename columns
municipality_mayor_dfs=municipality_mayor_dfs.rename(columns={"Muni Code":"Muni_Code","Muni Name":"Muni_Name",
                                                             "Mayor Name":"Mayor_Name","Term End":"Term_End"})

In [43]:
#Dispaly results - Final df
municipality_mayor_dfs.head(5)

,Muni_Code,Muni_Name,Mayor_Name,eMail,Address,Phone,Town,County,Term_End
0,101,Absecon City,KIMBERLY HORTON,khorton@abseconnj.org,"Absecon Municipal Complex, Absecon NJ -08201",(609) 641-0663,Absecon,Atlantic,2020-12-31
1,102,Atlantic City,"FRANK M. GILLIAM, JR.",pgeletei@CityofatlanticCity.org,"City Hall, Atlantic City NJ -08401",(609) 347-5300,Atlantic City,Atlantic,2021-12-31
2,103,Brigantine City,ANDREW SIMPSON,asimpson@brigantinebeachnj.com,"1417 West Brigantine Avenue, Brigantine NJ -08...",(609) 266-7600,Brigantine,Atlantic,2022-12-31
3,105,Buena Vista Township,Teresa Kelly,tkelly@buenavistanj.com,"890 Harding Highway, Buena NJ -08310",(856) 697-2100,Buena Vista,Atlantic,2021-12-31
4,106,Corbin City,ROBERT SCHULTE,mayor@corbinCitynj.com,"316 Route 50, Corbin City NJ -08270",(609) 628-2673,Corbin City,Atlantic,2020-12-31


In [44]:
#List columns to create db table
municipality_mayor_dfs.columns

Index(['Muni_Code', 'Muni_Name', 'Mayor_Name', 'eMail', 'Address', 'Phone',
       'Town', 'County', 'Term_End'],
      dtype='object')

### DB Schema

![images/QuickDBD-export.png](images/QuickDBD-export.png)


## Data Load - Insert data in to Database tables

In [3]:
#Check tables
engine.table_names()

['Senators',
 'Dist_Municipality',
 'MunicipalMayors',
 'SenatorAddress',
 'Assembly',
 'AssemblyPersonAddress']

In [47]:
#Use pandas to load csv converted DataFrame into database - Municipal df-1
municipality_mayor_dfs.to_sql('MunicipalMayors', con=engine, if_exists='append', index=False)

In [48]:
#Use pandas to load csv converted DataFrame into database - senator_legislators_df -2
senator_legislators_df.to_sql(name='Senators', con=engine, if_exists='append', index=False)

In [49]:
#Use pandas to load csv converted DataFrame into database - seneate_address_df-3
seneate_address_df.to_sql(name='SenatorAddress', con=engine, if_exists='append', index=False)

In [50]:
#Use pandas to load csv converted DataFrame into database - dist_municipality_dfs-4
dist_municipality_dfs.to_sql(name='Dist_Municipality', con=engine, if_exists='append', index=False)

In [51]:
#Use pandas to load csv converted DataFrame into database -assembly_address_df-5
assembly_address_df.to_sql(name='AssemblyPersonAddress', con=engine, if_exists='append', index=False)

In [52]:
#Use pandas to load csv converted DataFrame into database -assembly_lesgislators_df-6
assembly_lesgislators_df.to_sql(name='Assembly', con=engine, if_exists='append', index=False)

#### ----------------------------------- End of DB insert-updates -------------------------------

## Let's find out who all are my legislators

In [3]:
#Function to execute queries
def getResults(uFlag, town):
    
    #Create db connection
    conn= engine.connect()
    
    try:
        
        #Check user query flag before checking town
        if uFlag.upper()=="M":
            #Execute query to find Mayor info
            mayor_query_param = mayor_query.format("'"+ town +"'")
            return conn.execute(mayor_query_param)
        elif uFlag.upper()=="S":
            #Execute query to find Senator info
            senator_query_param = senator_query.format("'"+ town +"'")
            return conn.execute(senator_query_param)
        elif uFlag.upper()=="A":
            #Execute query to find Assembly info
            assembly_query_param = assembly_query.format("'"+ town +"'")
            return conn.execute(assembly_query_param)
        else:
            return "User entered invalid Flag. Valid query Flags -[M]/[S]/[A]"        
    except:
        return "Error occured while searching results. Please contact administration for the help."
    else:
        conn.close()

### User Intereaction - Search the results

In [4]:
#Continue Flag
bContinue = True

In [ ]:
#Loop through till - user quites. Note: To repeate again, please excute previous cell 
while bContinue:
    user_flag = input("\nWould you like to search legislator info of your town [Y/N]? ")
    if user_flag.strip().upper() =="Y":
        user_town = input("Please enter your township name [Example: Edison, South Brunswick]? ")
        user_choice = input("Please enter [M]ayor or [S]enator or [A]ssembly. Choices-[M/S/A]? ")
        
        #Check user choice on what to search
        if user_choice.upper()=="M":
            try:
                #Execute query to find Mayor info
                mayor_info= getResults("M",user_town.title())

                #Loop through results
                print("\n","-"*20,"Mayor Information","-"*20,"\n")
                for row in mayor_info:
                    info =list(row)
                    print("Name: ", info[0].strip())
                    print("eMail: ", info[1].strip())
                    print("Phone: ", info[3].strip())
                    print("Address: ", info[2].strip())
                print("\n", "-"*50,"\n")
            except:
                print("In-Error occured while searching results. Please contact administration for the help.")
                bContinue=False
                      
        elif user_choice.upper()=="S":
            try:
                #Execute query to find Senator info
                senator_info= getResults("S",user_town.title())

                #Loop through results
                print("\n","-"*20,"State Senator Information","-"*20,"\n")
                for row in senator_info:
                    info =list(row)
                    print("Name: ", info[0].strip())
                    print("Address/Phone: ", info[1].strip())
                print("\n", "-"*50,"\n")
            except:
                    print("In-Error occured while searching results. Please contact administration for the help.")
                    bContinue=False
                      
        elif user_choice.upper()=="A":
            try:
                #Execute query to find Assembly info
                assembly_info= getResults("A",user_town.title())

                #Loop through results
                print("\n","-"*20,"State Assembly Information","-"*20,"\n")
                for row in assembly_info:
                    info =list(row)
                    print("Name: ", info[0].strip())
                    print("Address/Phone: ", info[1].strip(),"\n")
                print("\n", "-"*50,"\n")
            except:
                    print("In-Error occured while searching results. Please contact administration for the help.")
                    bContinue=False
                      
        else:
            
            print("You selected invalid Flag for search. Valid query Flags -[M]/[S]/[A]")
            bContinue=False
    else:
        bContinue=False


Would you like to search legislator info of your town [Y/N]? y
Please enter your township name [Example: Edison, South Brunswick]? east brunswick
Please enter [M]ayor or [S]enator or [A]ssembly. Choices-[M/S/A]? m

 -------------------- Mayor Information -------------------- 

Name:  BRAD J. COHEN
eMail:  mayor@eastbrunswick.org
Phone:  (732) 390-6810
Address:  One Jean Walling Civic Center, East Brunswick NJ -08816-1081

 -------------------------------------------------- 


Would you like to search legislator info of your town [Y/N]? y
Please enter your township name [Example: Edison, South Brunswick]? east brunswick
Please enter [M]ayor or [S]enator or [A]ssembly. Choices-[M/S/A]? s

 -------------------- State Senator Information -------------------- 

Name:  PATRICK J. DIEGNAN, JR.
Address/Phone:  908 Oak Tree Ave., Unit P, South Plainfield, NJ 07080 (908) 757-1677

 -------------------------------------------------- 


Would you like to search legislator info of your town [Y/N]?